# 🚀 Spark Setup + Student DataFrame Practical (Jupyter)
This notebook:
1) Verifies environment (Python, `JAVA_HOME`, `SPARK_HOME`)
2) Initializes `findspark` and starts a `SparkSession`
3) Runs the **Student table** practical (DataFrame + SQL)
4) Demonstrates common DataFrame operations
5) Shows a mini ETL (Parquet write/read)
6) Covers grouping/aggregations, joins, complex types
7) (Optional) Transaction analytics mini-lab

> Run each section top-to-bottom. Comments and logs are included for clarity.

## 1. Environment Check

In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Utility: simple timestamped logger for clean outputs
import datetime
def log(msg):
    now = datetime.datetime.now().strftime('%H:%M:%S')
    print(f"[LOG {now}] {msg}")
log('Logger ready.')

[LOG 17:26:16] Logger ready.


In [3]:
# Check Python version and critical env vars
import sys, os
log(f'Python version: {sys.version.split()[0]}')
log(f'JAVA_HOME={os.environ.get("JAVA_HOME")}')
log(f'SPARK_HOME={os.environ.get("SPARK_HOME")}')

[LOG 17:26:17] Python version: 3.11.7
[LOG 17:26:17] JAVA_HOME=C:\Program Files\Java\jdk-17
[LOG 17:26:17] SPARK_HOME=None


## 2. Initialize Spark (findspark)

In [4]:
# If Spark isn't auto-detected, set SPARK_HOME explicitly before findspark.init()
# Example (uncomment and edit):
# import os
# os.environ['SPARK_HOME'] = r'C:\\spark'  # or '/usr/local/opt/spark'
import findspark
findspark.init()
log('findspark initialized.')

[LOG 17:26:17] findspark initialized.


## 3. Create SparkSession

In [5]:
# Create a Spark session with a small shuffle partition count for local runs
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName('Spark-Practical-Notebook')
         .config('spark.sql.shuffle.partitions', '4')
         .getOrCreate())
log(f'Spark started. Version: {spark.version}')
spark

[LOG 17:26:36] Spark started. Version: 4.0.0


## 4. Student Table: Schema + DataFrame + Temp View

In [ ]:


# Define schema explicitly for clarity and type safety
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('attendance', IntegerType(), True),
    StructField('score', IntegerType(), True),
    StructField('events', IntegerType(), True),
    StructField('projects', IntegerType(), True)
])

data = [
    (1, 'Amit',   92, 88, 3, 1),
    (2, 'Priya',  85, 91, 2, 2),
    (3, 'Rahul',  70, 76, 1, 0),
    (4, 'Neha',   88, 82, 2, 1),
    (5, 'Ravi',   90, 95, 3, 3),
    (6, 'Simran', 78, 89, 1, 1)
]

df = spark.createDataFrame(data, schema)
log('Student DataFrame created.')
df.show()
df.printSchema()

# Create temp view for Spark SQL queries
df.createOrReplaceTempView('students')
log('Temp view `students` is ready.')

[LOG 17:26:39] Student DataFrame created.
root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- attendance: integer (nullable = true)
 |-- score: integer (nullable = true)
 |-- events: integer (nullable = true)
 |-- projects: integer (nullable = true)

[LOG 17:26:40] Temp view `students` is ready.


## 5. Core Data Operations (select, filter, withColumn, orderBy)

In [7]:
from pyspark.sql import functions as F
# 5.1 Simple select
df.select('name', 'score', 'attendance').show


# 5.2 Filter rows
df.filter(F.col('attendance') > 85).show

# 5.3 New derived column
df2 = df.withColumn('score_boosted', F.col('score') + 2)
df2.orderBy(F.desc('score_boosted')).show
log('Core data operations complete.')

[LOG 17:26:41] Core data operations complete.


## 6. Spark SQL Examples

In [8]:
# Highest attendance
spark.sql('SELECT * FROM students ORDER BY attendance DESC LIMIT 1').show
# Names with score > 90
spark.sql('SELECT name FROM students WHERE score > 90').show
log('SQL queries executed.')

[LOG 17:26:41] SQL queries executed.


## 7. Basic ETL: Write/Read Parquet (local demo)

In [9]:
out_path = '/mnt/data/students_parquet'
df.write.mode('overwrite').parquet(out_path)
log(f'Wrote DataFrame to {out_path}')

back = spark.read.parquet(out_path)
log('Read back Parquet file:')
back.show()

Py4JJavaError: An error occurred while calling o76.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:314)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:1116)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:798)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:838)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:810)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:837)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs

## 8. Grouping & Aggregations

In [ ]:
# Overall aggregations
(df.groupBy()
   .agg(F.avg('score').alias('avg_score'), F.max('attendance').alias('max_attendance'))
   .show)

# Group by projects
(df.groupBy('projects')
   .agg(F.count('*').alias('n'), F.avg('score').alias('avg_score'))
   .orderBy('projects')
   .show)
log('Grouping & aggregations done.')

[LOG 11:53:20] Grouping & aggregations done.


## 9. Relational Operations (Joins)

In [ ]:
# Create a small lookup to join with students
sections = spark.createDataFrame([
    (1, 'A'), (2, 'B'), (3, 'A'), (5, 'C')
], ['id', 'section'])

joined = df.join(sections, on='id', how='left')
joined.show
log('Left join with sections completed.')

[LOG 11:53:43] Left join with sections completed.


## 10. Working with Complex Types (arrays, structs)

In [ ]:
complex_df = (df
    .withColumn('scores', F.array('score', 'attendance', 'events'))
    .withColumn('profile', F.struct('name', 'projects'))
)
complex_df.select('id', 'scores', 'profile').show(truncate=False)
log('Complex type demo done.')

Py4JJavaError: An error occurred while calling o353.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 31) (DARKKNIGHT executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed). Consider setting 'spark.sql.execution.pyspark.udf.faulthandler.enabled' or'spark.python.worker.faulthandler.enabled' configuration to 'true' for the better Python traceback.
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:621)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:599)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:35)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:945)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:925)
	at org.apache.spa

## 11. Mini-Lab: Transaction Analytics (replace with your real CSV path)
- Load a transactions CSV with headers
- Clean/cast numeric/date fields
- Compute daily totals and top products
- Save results as Parquet

In [ ]:
# Example path; change to your dataset path
path = '/mnt/data/sample_transactions.csv'  # e.g., 'C:/data/transactions.csv'
try:
    trans = (spark.read
             .option('header', True)
             .option('inferSchema', True)
             .csv(path))
    log(f'Loaded transactions: {trans.count()} rows')
    trans.printSchema()

    # Casting and simple KPIs
    trans2 = (trans
              .withColumn('amount', F.col('amount').cast('double'))
              .withColumn('transaction_date', F.to_date('transaction_date')))

    daily = (trans2.groupBy('transaction_date')
                    .agg(F.sum('amount').alias('total_amount')))
    daily.show(10)

    top_products = (trans2.groupBy('product')
                           .agg(F.sum('amount').alias('revenue'))
                           .orderBy(F.desc('revenue')))
    top_products.show(10)

    # Save outputs
    out_kpi = '/mnt/data/transaction_kpi_parquet'
    daily.write.mode('overwrite').parquet(out_kpi)
    log(f'Saved daily KPI to {out_kpi}')
except Exception as e:
    log(f'[Mini-Lab] Skipped or failed: {e}')

[LOG 11:55:43] [Mini-Lab] Skipped or failed: [PATH_NOT_FOUND] Path does not exist: file:/mnt/data/sample_transactions.csv. SQLSTATE: 42K03


## 12. Cleanup

In [ ]:
spark.stop()
log('Spark session stopped. ✅')

[LOG 11:55:54] Spark session stopped. ✅
